In [1]:
#Dependencies
import pandas as pd
from sqlalchemy import create_engine, inspect
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
#File with sql password and db name to be used as variables during connection
from sql_keys import sql_pass, sql_db

In [2]:
#Reading in vaccine_symptoms CSV
vaccine_symp = "data/vaccine_symptoms.csv"
vaccine_symp_df = pd.read_csv(vaccine_symp)
vaccine_symp_df.head()

,VAERS_ID,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
0,916710,Appendicitis,23.1,Band neutrophil percentage increased,23.1,Surgery,23.1,White blood cell count increased,23.1,NaN,NaN
1,916720,Dysphagia,23.1,Dyspnoea,23.1,NaN,NaN,NaN,NaN,NaN,NaN
2,916741,Chills,23.1,Complex regional pain syndrome,23.1,Fatigue,23.1,Headache,23.1,Joint range of motion decreased,23.1
3,916741,Myalgia,23.1,Pain in extremity,23.1,Peripheral swelling,23.1,X-ray abnormal,23.1,NaN,NaN
4,916772,Feeling abnormal,23.1,SARS-CoV-2 test positive,23.1,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#Reading in vaccine_type CSV
vaccine_type = "data/vaccine_type.csv"
vaccine_type_df = pd.read_csv(vaccine_type)
vaccine_type_df.head()

,VAERS_ID,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
0,916710,COVID19,MODERNA,NaN,1,IM,LA,COVID19 (COVID19 (MODERNA))
1,916720,COVID19,MODERNA,011L20A,1,IM,AR,COVID19 (COVID19 (MODERNA))
2,916741,COVID19,PFIZER\BIONTECH,EH9899,1,SYR,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
3,916772,COVID19,PFIZER\BIONTECH,EJ1685,UNK,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
4,916790,COVID19,PFIZER\BIONTECH,NaN,1,IM,RA,COVID19 (COVID19 (PFIZER-BIONTECH))


In [4]:
#Reading in vaccine_data CSV
vaccine_data = "data/vaccine_data.csv"
vaccine_df = pd.read_csv(vaccine_data,encoding= 'iso-8859-1')
vaccine_df.head(5)

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
0,916710,01/01/2021,MO,23.0,23.0,NaN,F,NaN,"Acute appendicitis, onset morning of 1/1/2021 ...",NaN,...,NaN,Hypothyroidism,NaN,NaN,2,01/01/2021,NaN,NaN,Y,NKDA
1,916720,01/01/2021,NY,23.0,23.0,NaN,M,NaN,Patient made statements that he was having a h...,NaN,...,NaN,NaN,unknown,NaN,2,01/01/2021,NaN,NaN,Y,NaN
2,916741,01/01/2021,AR,68.0,68.0,NaN,F,NaN,"on dec 22 I felt some myalgias, chills, fatigu...",NaN,...,had surgery R hand for advanced arthritis 11/1...,Rheumatoid arthritis - mostly affecting R wris...,NaN,NaN,2,01/01/2021,NaN,NaN,NaN,bee stings
3,916772,01/01/2021,GA,55.0,55.0,NaN,M,NaN,Vaccine on 12/22/2020 and started feeling bad ...,NaN,...,NaN,"HTN, Insomnia,High Cholesterol,",NaN,NaN,2,01/01/2021,NaN,Y,NaN,Codeine
4,916790,01/01/2021,TN,52.0,52.0,NaN,F,NaN,"Flushing, sweating, increased heart rate proce...",NaN,...,NaN,"Asthma, migraines",NaN,NaN,2,01/01/2021,NaN,NaN,Y,"Sulfa, shellfish"


In [5]:
#Creating a variable for the columns that are being used and creating a new dataframe
vaccine_data_cols = ["VAERS_ID", "RECVDATE", "AGE_YRS", "SEX", "DIED", "DATEDIED", "L_THREAT", "HOSPITAL", "HOSPDAYS", "RECOVD", "VAX_DATE", "HISTORY"]
vaccine_data_transformed= vaccine_df[vaccine_data_cols].copy()
#Changing the names of the columns
vaccine_data_transformed = vaccine_data_transformed.rename(columns={"VAERS_ID":"vaers_id", "RECVDATE": "recvdate",
                                                                    "AGE_YRS":"age_yrs", "SEX":"sex",
                                                                     "DIED":"died", "DATEDIED":"datedied",
                                                                   "L_THREAT":"l_threat", "HOSPITAL":"hospital", "HOSPDAYS":"hospdays", 
                                                                    "RECOVD":"recovd", "VAX_DATE":"vax_date", "HISTORY":"history"})
#Dropped duplicates from vaers_id column and set it as the index
vaccine_data_transformed.drop_duplicates("vaers_id", inplace=True)
vaccine_data_transformed.set_index("vaers_id", inplace=True)


vaccine_data_transformed.head(15)

,recvdate,age_yrs,sex,died,datedied,l_threat,hospital,hospdays,recovd,vax_date,history
vaers_id,,,,,,,,,,,
916710,01/01/2021,23.00,F,NaN,NaN,Y,Y,NaN,U,12/29/2020,Hypothyroidism
916720,01/01/2021,23.00,M,NaN,NaN,NaN,NaN,NaN,Y,12/31/2020,NaN
916741,01/01/2021,68.00,F,NaN,NaN,NaN,NaN,NaN,N,12/21/2020,Rheumatoid arthritis - mostly affecting R wris...
916772,01/01/2021,55.00,M,NaN,NaN,NaN,NaN,NaN,U,12/22/2020,"HTN, Insomnia,High Cholesterol,"
916790,01/01/2021,52.00,F,NaN,NaN,Y,NaN,NaN,U,12/26/2020,"Asthma, migraines"
916803,01/01/2021,78.00,M,Y,10/18/2020,Y,Y,8.0,N,10/06/2020,"large T-cell lymphoma, HTN, Gout, recieving t..."
916809,01/01/2021,40.00,F,NaN,NaN,NaN,Y,9.0,U,12/23/2020,NaN
916836,01/01/2021,55.00,M,NaN,NaN,NaN,Y,1.0,Y,12/24/2020,None
916859,01/01/2021,37.00,F,NaN,NaN,Y,NaN,NaN,Y,12/31/2020,"Undifferentiated connective tissue disorder, a..."


In [6]:
#Creating a variable for the columns that are being used and creating a new dataframe
vaccine_type_cols = ["VAERS_ID", "VAX_TYPE", "VAX_MANU", "VAX_DOSE_SERIES", "VAX_ROUTE", "VAX_NAME"]
vaccine_type_transformed= vaccine_type_df[vaccine_type_cols].copy()
#Changing the names of the columns
vaccine_type_transformed = vaccine_type_transformed.rename(columns={"VAERS_ID":"vaers_id", "VAX_TYPE": "vax_type",
                                                                    "VAX_MANU":"vax_manu", "VAX_DOSE_SERIES":"vax_dose_series",
                                                                    "VAX_ROUTE":"vax_route", "VAX_NAME":"vax_name"})
#Dropped duplicates from vaers_id column and set it as the index
vaccine_type_transformed.drop_duplicates("vaers_id", inplace=True)
vaccine_type_transformed.set_index("vaers_id", inplace=True)

#Filtered out the vax_types that weren't COVID19
vaccine_type_transformed=vaccine_type_transformed[vaccine_type_transformed['vax_type'] == 'COVID19']
vaccine_type_transformed.head()

,vax_type,vax_manu,vax_dose_series,vax_route,vax_name
vaers_id,,,,,
916710,COVID19,MODERNA,1,IM,COVID19 (COVID19 (MODERNA))
916720,COVID19,MODERNA,1,IM,COVID19 (COVID19 (MODERNA))
916741,COVID19,PFIZER\BIONTECH,1,SYR,COVID19 (COVID19 (PFIZER-BIONTECH))
916772,COVID19,PFIZER\BIONTECH,UNK,IM,COVID19 (COVID19 (PFIZER-BIONTECH))
916790,COVID19,PFIZER\BIONTECH,1,IM,COVID19 (COVID19 (PFIZER-BIONTECH))


In [7]:
#Creating a variable for the columns that are being used and creating a new dataframe
vaccine_symp_cols = ["VAERS_ID", "SYMPTOM1", "SYMPTOM2", "SYMPTOM3", "SYMPTOM4", "SYMPTOM5"]
vaccine_symp_transformed= vaccine_symp_df[vaccine_symp_cols].copy()
#Changing the names of the columns
vaccine_symp_transformed = vaccine_symp_transformed.rename(columns={"VAERS_ID":"vaers_id", "SYMPTOM1": "symptom1",
                                                                    "SYMPTOM2":"symptom2", "SYMPTOM3":"symptom3",
                                                                    "SYMPTOM4":"symptom4", "SYMPTOM5":"symptom5"})
#Dropped duplicates from vaers_id column 
vaccine_symp_transformed.drop_duplicates("vaers_id", inplace=True)


vaccine_symp_transformed

,vaers_id,symptom1,symptom2,symptom3,symptom4,symptom5
0,916710,Appendicitis,Band neutrophil percentage increased,Surgery,White blood cell count increased,NaN
1,916720,Dysphagia,Dyspnoea,NaN,NaN,NaN
2,916741,Chills,Complex regional pain syndrome,Fatigue,Headache,Joint range of motion decreased
4,916772,Feeling abnormal,SARS-CoV-2 test positive,NaN,NaN,NaN
5,916790,Blood pressure diastolic increased,Blood pressure systolic increased,Chest discomfort,Dizziness,Dysphagia
...,...,...,...,...,...,...
8632,1024788,Death,NaN,NaN,NaN,NaN
8633,1024790,Death,NaN,NaN,NaN,NaN
8634,1024795,Death,NaN,NaN,NaN,NaN
8635,1024817,Aspiration,Blood pressure systolic decreased,Cardio-respiratory arrest,Death,Dialysis


In [8]:
#Combined all the 5 symptoms in the vaccine_symp_transformed df into one column
all_symp = vaccine_symp_transformed.melt('vaers_id',["symptom1","symptom2","symptom3","symptom4","symptom5"])
#Renamed the value column
all_symp = all_symp.rename(columns={'value':'symptoms'})
#Made new df with only vaers_id and symptoms columns, set index to vaers_id and dropped NaN columns
all_symp = all_symp[['vaers_id','symptoms']]
all_symp.set_index("vaers_id", inplace=True)
all_symp.dropna(inplace=True)

all_symp

,symptoms
vaers_id,
916710,Appendicitis
916720,Dysphagia
916741,Chills
916772,Feeling abnormal
916790,Blood pressure diastolic increased
...,...
1024539,SARS-CoV-2 test positive
1024627,Disseminated intravascular coagulation
1024731,Nausea


In [9]:
#Created connection to the PosgreSQL DataBase
#Instead of having manualling entering pw, linked pw and db name to another file
connection_string = f"postgres:{sql_pass}@localhost:5432/{sql_db}"
engine = create_engine(f'postgresql://{connection_string}')

In [10]:
#Showing all the table names in DataBase
engine.table_names()

['vaccine_type', 'all_symp', 'vaccine_data']

In [11]:
#Reading the all_symp DataFrame into all_symp table
all_symp.to_sql(name='all_symp', con=engine, if_exists='replace', index=True)

In [12]:
#Reading the vaccine_type_transformed DataFrame into vaccine_type table
vaccine_type_transformed.to_sql(name='vaccine_type', con=engine, if_exists='replace', index=True)

In [13]:
#Reading the vaccine_data_transformed DataFrame into vaccine_data table
vaccine_data_transformed.to_sql(name='vaccine_data', con=engine, if_exists='replace', index=True)